---
title: "A Replication of Karlan and List (2007)"
author: "Dingran Wang"
date: 04/21/2025
callout-appearance: minimal # this hides the blue "i" icon on .callout-notes
---


## Introduction

Dean Karlan at Yale and John List at the University of Chicago conducted a field experiment to test the effectiveness of different fundraising letters. They sent out 50,000 fundraising letters to potential donors, randomly assigning each letter to one of three treatments: a standard letter, a matching grant letter, or a challenge grant letter. They published the results of this experiment in the _American Economic Review_ in 2007. The article and supporting data are available from the [AEA website](https://www.aeaweb.org/articles?id=10.1257/aer.97.5.1774) and from Innovations for Poverty Action as part of [Harvard's Dataverse](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/27853&version=4.2).

To explore how different fundraising strategies influence donor behavior, Karlan and List leveraged a natural field experiment in collaboration with a real nonprofit organization. The 50,000 prior donors were randomly assigned to receive one of several versions of a direct mail solicitation. The control group received a standard appeal, while treatment groups received letters offering matching grants at different match ratios—1:1, 2:1, or 3:1. Each variation also manipulated the suggested donation amount and the maximum size of the matching gift. By analyzing the response rate and donation amounts across treatments, the authors were able to isolate the causal impact of these fundraising tactics on charitable giving.

This project seeks to replicate their results.


## Data

### Description

In [ ]:
import pandas as pd

df = pd.read_stata("/Users/danielwang/Desktop/UCSD Spring/MGTA495 Marketing Analytics/Website/data/karlan_list_2007.dta")

:::: {.callout-note collapse="true"}
### Variable Definitions

| Variable             | Description                                                         |
|----------------------|---------------------------------------------------------------------|
| `treatment`          | Treatment                                                           |
| `control`            | Control                                                             |
| `ratio`              | Match ratio                                                         |
| `ratio2`             | 2:1 match ratio                                                     |
| `ratio3`             | 3:1 match ratio                                                     |
| `size`               | Match threshold                                                     |
| `size25`             | \$25,000 match threshold                                            |
| `size50`             | \$50,000 match threshold                                            |
| `size100`            | \$100,000 match threshold                                           |
| `sizeno`             | Unstated match threshold                                            |
| `ask`                | Suggested donation amount                                           |
| `askd1`              | Suggested donation was highest previous contribution                |
| `askd2`              | Suggested donation was 1.25 x highest previous contribution         |
| `askd3`              | Suggested donation was 1.50 x highest previous contribution         |
| `ask1`               | Highest previous contribution (for suggestion)                      |
| `ask2`               | 1.25 x highest previous contribution (for suggestion)               |
| `ask3`               | 1.50 x highest previous contribution (for suggestion)               |
| `amount`             | Dollars given                                                       |
| `gave`               | Gave anything                                                       |
| `amountchange`       | Change in amount given                                              |
| `hpa`                | Highest previous contribution                                       |
| `ltmedmra`           | Small prior donor: last gift was less than median \$35              |
| `freq`               | Number of prior donations                                           |
| `years`              | Number of years since initial donation                              |
| `year5`              | At least 5 years since initial donation                             |
| `mrm2`               | Number of months since last donation                                |
| `dormant`            | Already donated in 2005                                             |
| `female`             | Female                                                              |
| `couple`             | Couple                                                              |
| `state50one`         | State tag: 1 for one observation of each of 50 states; 0 otherwise  |
| `nonlit`             | Nonlitigation                                                       |
| `cases`              | Court cases from state in 2004-5 in which organization was involved |
| `statecnt`           | Percent of sample from state                                        |
| `stateresponse`      | Proportion of sample from the state who gave                        |
| `stateresponset`     | Proportion of treated sample from the state who gave                |
| `stateresponsec`     | Proportion of control sample from the state who gave                |
| `stateresponsetminc` | stateresponset - stateresponsec                                     |
| `perbush`            | State vote share for Bush                                           |
| `close25`            | State vote share for Bush between 47.5% and 52.5%                   |
| `red0`               | Red state                                                           |
| `blue0`              | Blue state                                                          |
| `redcty`             | Red county                                                          |
| `bluecty`            | Blue county                                                         |
| `pwhite`             | Proportion white within zip code                                    |
| `pblack`             | Proportion black within zip code                                    |
| `page18_39`          | Proportion age 18-39 within zip code                                |
| `ave_hh_sz`          | Average household size within zip code                              |
| `median_hhincome`    | Median household income within zip code                             |
| `powner`             | Proportion house owner within zip code                              |
| `psch_atlstba`       | Proportion who finished college within zip code                     |
| `pop_propurban`      | Proportion of population urban within zip code                      |

::::


### Balance Test 

As an ad hoc test of the randomization mechanism, I provide a series of tests that compare aspects of the treatment and control groups to assess whether they are statistically significantly different from one another.

In [ ]:
import statsmodels.api as sm
from scipy import stats

# Load data
df = pd.read_stata("karlan_list_2007.dta")
df['treat'] = df['treatment'].fillna(0)

# Balance test function
def balance_test(var):
    df_sub = df[['treat', var]].dropna()
    treat_group = df_sub[df_sub['treat'] == 1][var]
    control_group = df_sub[df_sub['treat'] == 0][var]
    
    # T-test
    t_stat, p_val = stats.ttest_ind(treat_group, control_group, equal_var=True)
    mean_diff = treat_group.mean() - control_group.mean()

    print(f"\n==== Balance Test for '{var}' ====")
    print(f"T-test:")
    print(f"  Mean (Treatment): {treat_group.mean():.3f}")
    print(f"  Mean (Control):   {control_group.mean():.3f}")
    print(f"  Difference:       {mean_diff:.3f}")
    print(f"  t-statistic:      {t_stat:.3f}")
    print(f"  p-value:          {p_val:.4f}")

    # OLS
    X = sm.add_constant(df_sub['treat'])
    model = sm.OLS(df_sub[var], X).fit()
    print("\nOLS Regression:")
    print(model.summary().tables[1])

# Run tests
for var in ['mrm2', 'freq', 'hpa']:
    balance_test(var)


## Experimental Results
Variable | Mean (Treatment) | Mean (Control) | Difference | t-stat | p-value | Significant?
mrm2 | 13.012 | 12.998 | +0.014 | 0.119 | 0.9049 | ❌
freq | 8.035 | 8.047 | -0.012 | -0.111 | 0.9117 | ❌
hpa | 59.597 | 58.960 | +0.637 | 0.944 | 0.3451 | ❌

In all cases, the null hypothesis of equal means cannot be rejected, and both t-tests and OLS regressions yield identical conclusions. The treatment assignment is statistically uncorrelated with these pre-treatment variables, providing strong evidence that the randomization mechanism worked as intended.

Why this matters: Table 1 in Karlan & List (2007) serves the same purpose—showing that groups were well-balanced at baseline. This is essential for internal validity: it ensures that any post-treatment differences in giving behavior can be credibly attributed to the treatment itself, not to pre-existing differences between donors.


### Charitable Contribution Made

First, I analyze whether matched donations lead to an increased response rate of making a donation. 

In [ ]:
import matplotlib.pyplot as plt

# Create treatment indicator
df['treat'] = df['treatment'].fillna(0)

# Group by treatment and compute mean of 'gave' (binary)
prop_donated = df.groupby('treat')['gave'].mean()

# Plot
plt.bar(['Control', 'Treatment'], prop_donated, edgecolor='black')
plt.title("Proportion of Donors by Treatment Group")
plt.ylabel("Proportion Who Donated")
plt.ylim(0, 0.03)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()

The barplot below shows that 2.20% of individuals in the treatment group donated, compared to 1.79% in the control group.


In [ ]:
# T-test
gave_treat = df[df['treat'] == 1]['gave']
gave_control = df[df['treat'] == 0]['gave']
t_stat, p_val = stats.ttest_ind(gave_treat, gave_control, equal_var=True)

print("T-test results:")
print(f"  Mean (Treatment): {gave_treat.mean():.4f}")
print(f"  Mean (Control):   {gave_control.mean():.4f}")
print(f"  Difference:       {gave_treat.mean() - gave_control.mean():.4f}")
print(f"  t-statistic:      {t_stat:.3f}")
print(f"  p-value:          {p_val:.4f}")

# OLS regression
X = sm.add_constant(df['treat'])
ols_model = sm.OLS(df['gave'], X).fit()
print("\nOLS Regression:")
print(ols_model.summary().tables[1])

T-test:
  Mean (Treatment): 0.0220
  Mean (Control):   0.0179
  Difference:       0.0042
  t-statistic:      3.101
  p-value:          0.0019

OLS Regression:
  Coefficient on treatment: 0.0042
  p-value:                  0.002

The t-test and regression agree: the difference is statistically significant at the 1% level. Offering a matching grant increased the response rate by about 0.42 percentage points, which is approximately a 22% increase relative to the control group's donation rate.


In [ ]:
# Probit model
probit_model = sm.Probit(df['gave'], X).fit()
print(probit_model.summary())

Probit Coefficient on treatment: 0.0868
z-statistic: 3.113
p-value:     0.002

This replicates Table 3, Column 1 in the original paper, confirming that the treatment has a positive and statistically significant effect on the probability of donating.

These results show that simply informing potential donors that their gift would be matched made them more likely to give. Even though the absolute increase in donation rate is small, the relative effect is large and meaningful for fundraisers.

This supports the idea that donors are motivated not only by altruism, but also by how effective or impactful their contribution feels. The matching grant may act as a psychological signal that "now is a good time to give" or that their donation is more valuable than usual.


### Differences between Match Rates

Next, I assess the effectiveness of different sizes of matched donations on the response rate.

In [ ]:
# 只对 treatment 组分析
treat_df = df[df['treat'] == 1].copy()

# 比较 1:1 vs 2:1
gave_1 = treat_df[treat_df['ratio'] == 1]['gave']
gave_2 = treat_df[treat_df['ratio'] == 2]['gave']
t12, p12 = stats.ttest_ind(gave_1, gave_2)

# 比较 2:1 vs 3:1
gave_3 = treat_df[treat_df['ratio'] == 3]['gave']
t23, p23 = stats.ttest_ind(gave_2, gave_3)

print(f"1:1 vs 2:1 match rate — p = {p12:.4f}")
print(f"2:1 vs 3:1 match rate — p = {p23:.4f}")
print(f"Means: 1:1 = {gave_1.mean():.4f}, 2:1 = {gave_2.mean():.4f}, 3:1 = {gave_3.mean():.4f}")

T-tests Between Match Ratios

1:1 vs 2:1 — p = 0.3345
2:1 vs 3:1 — p = 0.9600
Means: 
  1:1 = 0.0207
  2:1 = 0.0226
  3:1 = 0.0227

The results show no statistically significant differences in donation rates between the match levels. While there is a slight increase in the mean from 1:1 to 2:1 and 3:1, the p-values (0.33 and 0.96) confirm that these differences are not distinguishable from zero at conventional significance levels.


In [ ]:
# 创建 dummy 变量
treat_df['ratio1'] = (treat_df['ratio'] == 1).astype(int)
treat_df['ratio2'] = (treat_df['ratio'] == 2).astype(int)
treat_df['ratio3'] = (treat_df['ratio'] == 3).astype(int)

# 用 ratio1 作为基准组
X = sm.add_constant(treat_df[['ratio2', 'ratio3']])
y = treat_df['gave']
ratio_model = sm.OLS(y, X).fit()
print(ratio_model.summary().tables[1])

Regression on Match Ratio Dummies

Regression of `gave` on ratio2 and ratio3 (baseline = 1:1):
ratio2 coefficient:  +0.0019 (p = 0.338)  
ratio3 coefficient:  +0.0020 (p = 0.313)

This regression supports the t-test findings. Neither the 2:1 nor 3:1 match ratio has a statistically significant effect relative to the 1:1 match. This aligns closely with Karlan & List’s statement that “larger match ratios… had no additional impact” (p. 8).


In [ ]:
# 直接从数据中计算
diff_21 = gave_2.mean() - gave_1.mean()
diff_32 = gave_3.mean() - gave_2.mean()

# 从 OLS 回归系数中计算
coef_diff_21 = ratio_model.params['ratio2']
coef_diff_32 = ratio_model.params['ratio3'] - ratio_model.params['ratio2']

print(f"Data-based diff (2:1 - 1:1): {diff_21:.4f}")
print(f"Data-based diff (3:1 - 2:1): {diff_32:.4f}")
print(f"Model-based diff (2:1 - 1:1): {coef_diff_21:.4f}")
print(f"Model-based diff (3:1 - 2:1): {coef_diff_32:.4f}")

Direct vs Model-Based Differences

Comparison | Data Difference | Model Coefficient Difference
2:1 – 1:1 | +0.0019 | +0.0019
3:1 – 2:1 | +0.0001 | +0.0001

Both direct comparisons and model-based coefficient differences tell the same story: moving from 1:1 to 2:1 yields a small and statistically insignificant increase, and going from 2:1 to 3:1 yields essentially no change at all.

These results suggest that offering a match increases the chance of donation, but increasing the match ratio further does not enhance this effect. In other words, once the donor sees their gift will be matched, the degree of matching is not very motivating. This supports the notion that framing and salience—not just raw incentive size—drive much of charitable behavior.

This provides valuable insights for fundraisers: even modest matching offers (1:1) may be just as effective as more expensive ones (2:1 or 3:1) in driving participation.


### Size of Charitable Contribution

In this subsection, I analyze the effect of the size of matched donation on the size of the charitable contribution.

T-test / Regression on Full Sample

In [ ]:
# 线性回归：所有人（包括未捐赠者）
X_full = sm.add_constant(df['treat'])
y_amount = df['amount']
model_full = sm.OLS(y_amount, X_full).fit()

print("OLS on all individuals:")
print(model_full.summary().tables[1])

Dependent variable: donation amount (all individuals)

Treatment coefficient: +0.154  
p-value: 0.063

This suggests that offering a matching grant increased the average donation amount by about $0.15. However, the result is only marginally significant (p ≈ 0.063). Since this regression includes non-donors (who gave $0), the result likely reflects the fact that more people gave at all in the treatment group.


Regression Conditional on Donation

In [ ]:
# 限制在实际有捐款的人
df_positive = df[df['gave'] == 1]

X_cond = sm.add_constant(df_positive['treat'])
y_cond_amount = df_positive['amount']
model_cond = sm.OLS(y_cond_amount, X_cond).fit()

print("\nOLS on donors only (conditional on giving):")
print(model_cond.summary().tables[1])

Dependent variable: donation amount (only donors)

Treatment coefficient: -1.668  
p-value: 0.561

Among those who actually donated, individuals in the treatment group gave slightly less, on average, than those in the control group—but this difference is small and statistically insignificant.

This suggests that the treatment did not affect how much people gave, once they decided to give. The treatment influenced the extensive margin (whether to donate), but not the intensive margin (how much to donate).

Causal note: Since we are conditioning on a post-treatment outcome (gave), the regression on donors only does not have a causal interpretation. It is, however, still descriptively valuable.



Histograms of Donation Amounts (Among Donors)

In [ ]:
import matplotlib.pyplot as plt

# 仅有捐赠的人，按 treatment 分组
donors_treat = df_positive[df_positive['treat'] == 1]['amount']
donors_ctrl = df_positive[df_positive['treat'] == 0]['amount']

# 画图设置
fig, ax = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# 控制组
ax[0].hist(donors_ctrl, bins=30, color='skyblue', edgecolor='black')
ax[0].axvline(donors_ctrl.mean(), color='red', linestyle='dashed', linewidth=2)
ax[0].set_title("Control Group: Donation Amounts")
ax[0].set_xlabel("Amount")
ax[0].set_ylabel("Number of Donors")

# 处理组
ax[1].hist(donors_treat, bins=30, color='lightgreen', edgecolor='black')
ax[1].axvline(donors_treat.mean(), color='red', linestyle='dashed', linewidth=2)
ax[1].set_title("Treatment Group: Donation Amounts")
ax[1].set_xlabel("Amount")

plt.suptitle("Histograms of Donation Amounts (Among Donors)", fontsize=14)
plt.tight_layout()
plt.show()

The following histograms show the distribution of donation amounts among donors only, with red dashed lines indicating group-specific average donations.

The treatment and control groups show similar right-skewed distributions, with most gifts clustered around $25–$75 and a few very large gifts (e.g., $250 or $400). The average donation was slightly higher in the control group, consistent with the regression results.



## Simulation Experiment

As a reminder of how the t-statistic behaves under repeated sampling, I simulate the process of comparing two independent groups with known probabilities of donating:

Control group: donation follows a Bernoulli distribution with probability p = 0.018

Treatment group: donation follows a Bernoulli distribution with probability p = 0.022

I simulate the sampling and testing process many times, and observe how the t-statistic behaves under different sample sizes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Parameters
p_control = 0.018
p_treat = 0.022
n_sims = 10000
sample_sizes = [100, 500, 1000, 5000]

# Function: simulate t-stats
def simulate_t_stats(n, sims=10000):
    t_stats = []
    for _ in range(sims):
        control = np.random.binomial(1, p_control, size=n)
        treat = np.random.binomial(1, p_treat, size=n)
        t_stat, _ = stats.ttest_ind(treat, control)
        t_stats.append(t_stat)
    return t_stats

# Run simulations
tstat_results = {n: simulate_t_stats(n) for n in sample_sizes}

In [ ]:
# Plot sampling distributions
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.ravel()

for i, n in enumerate(sample_sizes):
    axes[i].hist(tstat_results[n], bins=40, density=True, color='skyblue', edgecolor='black')
    axes[i].axvline(x=0, color='red', linestyle='--')
    axes[i].set_title(f"Sampling Distribution of t-stat (n = {n})")
    axes[i].set_xlabel("t-statistic")
    axes[i].set_ylabel("Density")

plt.suptitle("Sampling Distribution of t-statistics under Bernoulli Model", fontsize=14)
plt.tight_layout()
plt.show()

Interpretation
When sample size is small (e.g., n = 100), the t-statistic distribution is wide and noisy — it's hard to tell if there’s a real effect.

As sample size increases, the t-statistic distribution becomes narrower, and centered around the true difference (which is small, but positive).

This reflects the Law of Large Numbers: sample means converge to the population means.

It also illustrates the Central Limit Theorem: the difference in two sample means becomes approximately normally distributed as sample size grows.

In other words: with small samples, we might miss a real effect due to noise. But as sample size increases, our estimate of the effect and our test statistics become more stable and reliable.


### Law of Large Numbers

Visualization: Cumulative Average of Differences

In [ ]:
# Simulate draws
np.random.seed(42)
control_draws = np.random.binomial(1, 0.018, size=100000)
treat_draws = np.random.binomial(1, 0.022, size=10000)

# Only use first 10,000 for matching length
control_sample = control_draws[:10000]

# Compute vector of differences (elementwise)
diffs = treat_draws - control_sample

# Compute cumulative average of differences
cum_avg = np.cumsum(diffs) / np.arange(1, len(diffs) + 1)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(cum_avg, label='Cumulative Average of Differences')
plt.axhline(0.004, color='red', linestyle='--', label='True Difference (0.004)')
plt.title("Law of Large Numbers: Cumulative Average of Treatment - Control")
plt.xlabel("Number of Samples")
plt.ylabel("Cumulative Average Difference")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

The plot below shows the cumulative average of the differences in donation outcomes between simulated treatment and control groups:


Initially, the average fluctuates wildly due to the high variance in small samples. For example, the first few differences swing far above and below the true mean.

As the number of samples increases, the cumulative average stabilizes and begins to hover around the true difference of 0.004, indicated by the red dashed line. This illustrates the Law of Large Numbers in practice: the more data we gather, the closer our estimate gets to the truth.

### Central Limit Theorem

In [ ]:
# Parameters
p_c = 0.018
p_t = 0.022
sample_sizes = [50, 200, 500, 1000]
sim_count = 1000

# Simulate average differences for each sample size
def simulate_avg_diffs(n, sims=1000):
    diffs = []
    for _ in range(sims):
        control = np.random.binomial(1, p_c, size=n)
        treatment = np.random.binomial(1, p_t, size=n)
        diffs.append(np.mean(treatment) - np.mean(control))
    return np.array(diffs)

# Run simulations
results_clt = {n: simulate_avg_diffs(n) for n in sample_sizes}

In [ ]:
# Plot histograms
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.ravel()

for i, n in enumerate(sample_sizes):
    diffs = results_clt[n]
    axes[i].hist(diffs, bins=40, color='lightblue', edgecolor='black')
    axes[i].axvline(0, color='red', linestyle='--', label='Zero (H₀)')
    axes[i].axvline(0.004, color='green', linestyle='-', label='True Difference')
    axes[i].set_title(f"Sample Size = {n}")
    axes[i].set_xlabel("Average Difference (Treatment - Control)")
    axes[i].set_ylabel("Frequency")
    axes[i].legend()

plt.suptitle("Sampling Distributions of Difference in Means\n(Central Limit Theorem)", fontsize=14)
plt.tight_layout()
plt.show()

Interpretation
At sample size = 50: The distribution is erratic, skewed, and shows wide dispersion. Zero is right in the center, meaning we likely wouldn't detect a small treatment effect.

At sample size = 200: The histogram becomes more symmetric and bell-shaped, and the true effect (green line) begins to emerge away from zero.

At sample size = 500: The distribution becomes much smoother and narrower. Zero starts moving toward the tail, suggesting a higher chance of rejecting the null.

At sample size = 1000: The distribution is tightly centered near the true difference of 0.004, and zero is well into the left tail—implying we would confidently detect the effect in most samples.

These plots visually confirm the Central Limit Theorem: as sample size increases, the sampling distribution of the sample mean difference:

Becomes more normal in shape

Narrows in spread (reduced variance)

Centers around the true mean difference
